In [ ]:
%load_ext Cython
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import Final
import numpy as np

# NUM_REGISTROS: Final[int] = 4.5 * 10**9
NUM_REGISTROS: Final[int] = 0.1 * 10**9
ARRAY_NUMEROS = np.arange(NUM_REGISTROS, dtype=np.uint32)

print(f"{ARRAY_NUMEROS.size = :,}")

In [ ]:
%%cython
cimport cython
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef unsigned long get_max_value(unsigned long[:] array):
    cdef Py_ssize_t index
    cdef Py_ssize_t array_size = array.size
    cdef unsigned long max_value = array[0]
    
    for index in range(1, array_size):
        if array[index] > max_value:
            max_value = array[index]
        
    return max_value

In [ ]:
from src import get_max_value_C

In [ ]:
get_max_value_C(ARRAY_NUMEROS)

In [ ]:
np.max(ARRAY_NUMEROS) == get_max_value_C(ARRAY_NUMEROS)

In [ ]:
# # Cython (Função em C)
# %timeit -p 5 get_max_value_C(ARRAY_NUMEROS)

# # Usando Numpy
# %timeit -p 5 np.max(ARRAY_NUMEROS)

# # Cython (Jupyter)
# %timeit -p 5 get_max_value(ARRAY_NUMEROS)

# # Implementação padrão
# %timeit -p 5 max(ARRAY_NUMEROS)

In [ ]:
# Cython (Função em C)
%timeit -r 1 -p 5 get_max_value_C(ARRAY_NUMEROS)
# Usando Numpy
%timeit -r 1 -p 5 np.max(ARRAY_NUMEROS)

In [ ]:
import time

t1 = time.perf_counter()
get_max_value_C(ARRAY_NUMEROS)
time.sleep(3)
t2 = time.perf_counter()
print(f"{t2 - t1 = }")

t1 = time.process_time()
get_max_value_C(ARRAY_NUMEROS)
time.sleep(3)
t2 = time.process_time()
print(f"{t2 - t1 = }")

In [ ]:
import timeit

timeit.repeat(
    "get_max_value_C(ARRAY_NUMEROS)",
    globals=globals(),
    repeat=1,
    number=1,
)